In [1]:
# This ipynb file is adapted from https://github.com/wagonhelm/TF_ObjectDetection_API/blob/master/ChessObjectDetection.ipynb

import skimage
import numpy as np
from skimage import io, transform
import os
import shutil
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import tensorflow as tf
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import urllib.request
import urllib.error

%matplotlib inline

Before opening the Jupyter Notebook make sure you have cloned the `models` folder into the repository root directory and run the following from the root diretory to install the TensorFlow API

```bash
git clone https://github.com/tensorflow/models.git
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
cd ..
cd ..
```

Set Up Path Directories
--------------------------

In [2]:
root = os.getcwd()
imagePath = os.path.join(root, 'chairs')
labelsPath = os.path.join(root, 'labels')
linksPath = os.path.join(imagePath, 'imageLinks')
trainPath = os.path.join(imagePath, 'train')
testPath = os.path.join(imagePath, 'test')

Convert XML Labels to CSV
-----------------------------

In [3]:
# Modified From:
# https://github.comr/datitran/raccoon_dataset/blob/master/xml_to_csv.py

os.chdir(root)

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    
    for i in [trainPath, testPath]:
        image_path = i
        folder = os.path.basename(os.path.normpath(i))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv('data/'+folder+'.csv', index=None)
        print('Successfully converted xml to csv.')
    
main()

Successfully converted xml to csv.
Successfully converted xml to csv.


Create TF Record
------------------------------------------------------

When training models with TensorFlow using [tfrecords](http://goo.gl/oEyYyR) files help optimize your data feed.  We can generate a tfrecord using code adapted from this [raccoon detector](https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py).

In [4]:
%%bash 

python generate_tfrecord.py
mv test.record data
mv train.record data

Traceback (most recent call last):
  File "generate_tfrecord.py", line 14, in <module>
    from object_detection.utils import dataset_util
ModuleNotFoundError: No module named 'object_detection'
mv: cannot stat 'test.record': No such file or directory
mv: cannot stat 'train.record': No such file or directory


CalledProcessError: Command 'b'\npython generate_tfrecord.py\nmv test.record data\nmv train.record data\n'' returned non-zero exit status 1.

Download Model
----------------

There are [models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) in the TensorFlow API that you can use depending on your needs.  If you want a high speed model that can work on detecting video feed at high fps the [single shot detection](http://www.cs.unc.edu/%7Ewliu/papers/ssd.pdf) model works best, but you gain speed at the cost of accuracy. Some object detection models detect objects by sliding different sized boxes across the image running the classifier many time on different sections of the image, this of course can be very resource consuming.  As it’s name suggests single shot detection determines all bounding box probabilities in one go, hence why it is a vastly faster model. I’ve already configured the [config](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs) file for mobilenet and included it in the GitHub repository for this post.  Depending on your computer you may have to lower the batch size in the config file if you run out of memory.



In [ ]:
%%bash

wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz
tar xvzf ssd_mobilenet_v1_coco_11_06_2017.tar.gz

Train Model
-------------
Since we are only retraining the last layer of our mobilenet model a high end gpu is not required (but certainly can speed things up). Training time should roughly take an hour.  It will be much easier to watch the training process if you copy and paste the following code into a new terminal in the repository root directory.  Once our loss drops to a consistant level for a good while we can stop TensorFlow training by pressing ctrl+c.

To train the model copy and paste the following code into a new terminal from the repository root directory.  If using Docker create a new terminal pressing `ctrl` + `b` then `c`.

```bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
cd ..
cd ..

python3 models/research/object_detection/train.py --logtostderr --train_dir=data/ --pipeline_config_path=data/ssd_mobilenet_v1_pets.config
```

Watch Training in TensorBoard
---------------------------------

We can use TensorBoard to monitor our total loss and other variables.  From the repository root directory run this command.

```bash
tensorboard --logdir='data'
```

Export Inference Graph
-------------------------

I highly recommend you expiriment with different checkpoints as your model trains.  We can get a list of all the ckpt files with the following.

In [ ]:
%%bash 
cd data
ls model*.index

You can then added the cpkt number to our trained_checkpoint argument.

In [ ]:
%%bash 
rm -rf object_detection_graph
python3 models/research/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path data/ssd_mobilenet_v1_pets.config \
    --trained_checkpoint_prefix data/model.ckpt-997 \
    --output_directory object_detection_graph

Test Model
-----------

In [ ]:
%%bash 
python3 test.py